* [#53 SBL dataset släpps 2024 Strings not Things](https://github.com/salgo60/Svenskaforsamlingar/issues/53)

This checks WD see also [webscraping](https://github.com/salgo60/open-data-examples/blob/master/SBL.ipynb)

In [1]:
import time 
from datetime import datetime
start_time = time.time() 
start_datetime = datetime.fromtimestamp(start_time) 
print("Start time:", start_datetime.strftime('%Y-%m-%d %H:%M:%S'))

Start time: 2025-11-29 05:53:19


In [8]:
import requests
import pandas as pd
from io import StringIO

# URL of the file to be fetched
url = "https://filer.riksarkivet.se/registerdata/SBL/csv/SBL_2023.csv"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Use StringIO to read the CSV content into a pandas DataFrame
    csv_content = StringIO(response.text)
    # Reset the StringIO object to start reading from the beginning
    csv_content.seek(0)
    
    # Read the CSV into a DataFrame using the correct delimiter
    df = pd.read_csv(csv_content, delimiter=';')  # Adjust the delimiter if needed

    
    df.to_csv("SBL_2023_local.csv", index=False)
    print("Data has been downloaded, loaded into pandas, and saved as SBL_2023_local.csv")
else:
    print(f"Failed to download file. Status code: {response.status_code}")


Data has been downloaded, loaded into pandas, and saved as SBL_2023_local.csv


## Status 
* 2024-05-14: Article id = 9406
* 2025-11-29: Article id = 9406

 

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9406 entries, 0 to 9405
Data columns (total 47 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Article id                              9406 non-null   int64  
 1   Svenskt biografiskt lexikon (SBL): URI  9406 non-null   object 
 2   Type of article                         9406 non-null   object 
 3   SBL volume number                       9405 non-null   float64
 4   Page number in volume                   9404 non-null   float64
 5   Surname                                 9406 non-null   object 
 6   Given name                              8949 non-null   object 
 7   Gender                                  9406 non-null   object 
 8   Occupation, royal title, rank           7604 non-null   object 
 9   Prefix to year of birth                 53 non-null     object 
 10  Year of birth                           7620 non-null   obje

## Get WD data

In [4]:
!pip3 install SPARQLWrapper


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [5]:
pip install --upgrade certifi

  Attempting uninstall: certifi
    Found existing installation: certifi 2025.8.3
    Uninstalling certifi-2025.8.3:
      Successfully uninstalled certifi-2025.8.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-server 2.25.3 requires requests>=2.31, but you have requests 2.26.0 which is incompatible.
selenium 4.35.0 requires urllib3[socks]<3.0,>=2.5.0, but you have urllib3 1.26.20 which is incompatible.
osmnx 2.0.6 requires requests>=2.27, but you have requests 2.26.0 which is incompatible.
instagrapi 2.2.1 requires requests==2.32.4, but you have requests 2.26.0 which is incompatible.
wikibaseintegrator 0.12.8 requires requests<3.0.0,>=2.31.0, but you have requests 2.26.0 which is incompatible.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.12/bin/python3.12 -m pip 

In [6]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT (REPLACE(STR(?item), ".*Q", "Q") AS ?wid)  ?sblid ?gender ?genderLabel WHERE {
?item wdt:P3217 ?sblid.
?item wdt:P31 wd:Q5.
optional {?item wdt:P21 ?gender}
SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }  
}"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    print(result)


{'sblid': {'type': 'literal', 'value': '12996'}, 'gender': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q6581097'}, 'genderLabel': {'xml:lang': 'sv', 'type': 'literal', 'value': 'man'}, 'wid': {'type': 'literal', 'value': 'Q71478'}}
{'sblid': {'type': 'literal', 'value': '13329'}, 'gender': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q6581097'}, 'genderLabel': {'xml:lang': 'sv', 'type': 'literal', 'value': 'man'}, 'wid': {'type': 'literal', 'value': 'Q250560'}}
{'sblid': {'type': 'literal', 'value': '20337'}, 'gender': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q6581097'}, 'genderLabel': {'xml:lang': 'sv', 'type': 'literal', 'value': 'man'}, 'wid': {'type': 'literal', 'value': 'Q327853'}}
{'sblid': {'type': 'literal', 'value': '17600'}, 'gender': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q6581097'}, 'genderLabel': {'xml:lang': 'sv', 'type': 'literal', 'value': 'man'}, 'wid': {'type': 'literal', 'value': 'Q481585'}}
{'sblid': {'type'

In [7]:
from SPARQLWrapper import SPARQLWrapper, JSON
import sys
endpoint_url = "https://query.wikidata.org/sparql"

#https://w.wiki/cvY
querySBL = """SELECT (REPLACE(STR(?item), ".*Q", "Q") AS ?wid)  ?sblid ?gender ?genderLabel WHERE {
?item wdt:P3217 ?sblid.
?item wdt:P31 wd:Q5.
optional {?item wdt:P21 ?gender}
SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }  
}"""

def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDSBLtot = get_sparql_dataframe(endpoint_url, querySBL)

NameError: name 'json' is not defined

In [ ]:
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print(f"Elapsed time: {elapsed_time} seconds")